In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
url = "http://quotes.toscrape.com/"

In [3]:
# Realizamos la petición HTTP
print("Conectando a la página web...")
response = requests.get(url)

Conectando a la página web...


In [4]:
# Verificamos que la petición fue exitosa
if response.status_code == 200:
    print("Conexión exitosa. Código de estado:", response.status_code)

    # Creamos el objeto BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    print("\n" + "="*50)
    print("FUNCIONALIDAD 1: BÚSQUEDA Y NAVEGACIÓN BÁSICA")
    print("="*50)

    # Ejemplo 1: Obtener el título de la página
    page_title = soup.title.text
    print(f"Título de la página: {page_title}")

    # Ejemplo 1.1: Encontrar elementos por nombre de etiqueta
    all_quotes = soup.find_all('div', class_='quote')
    print(f"\nCitas encontradas: {len(all_quotes)}")

    # Mostramos las primeras 3 citas
    print("\nPrimeras 3 citas:")
    for i, quote in enumerate(all_quotes[:3], 1):
        quote_text = quote.find('span', class_='text').get_text(strip=True)
        print(f"{i}. {quote_text}")

    # Ejemplo 1.2: Buscar el primer elemento que coincida
    first_author = soup.find('small', class_='author')
    print(f"\nPrimer autor encontrado: {first_author.text}")

    print("\n" + "="*50)
    print("FUNCIONALIDAD 2: BÚSQUEDA AVANZADA CON SELECTORES CSS Y FILTROS")
    print("="*50)

    # Ejemplo 2.1: Usar selectores CSS
    quotes = soup.select('div.quote')
    print(f"Número de citas encontradas con CSS selector: {len(quotes)}")

    # Ejemplo 2.2: Búsqueda con filtros personalizados
    # Buscar citas de autores específicos
    def author_filter(tag):
        if tag.name == 'small' and tag.has_attr('class') and 'author' in tag['class']:
            author_text = tag.text.strip()
            # Buscamos citas de Albert Einstein
            return author_text == "Albert Einstein"
        return False

    einstein_quotes = soup.find_all(author_filter)
    print(f"\nCitas de Albert Einstein: {len(einstein_quotes)}")

    # Ejemplo 2.3: Filtrar por texto contenido
    love_quotes = soup.find_all(text=lambda text: 'love' in text.lower() if text else False)
    print(f"\nCitas que contienen la palabra 'love': {len(love_quotes)}")

    print("\n" + "="*50)
    print("FUNCIONALIDAD 3: EXTRACCIÓN Y MANIPULACIÓN DE ATRIBUTOS")
    print("="*50)

    # Ejemplo 3.1: Extraer enlaces y atributos
    author_links = []

    for quote in quotes[:5]:  # Tomamos las primeras 5 citas
        # Obtener el texto de la cita
        quote_text = quote.find('span', class_='text').get_text(strip=True)

        # Obtener el autor
        author = quote.find('small', class_='author').get_text(strip=True)

        # Obtener el enlace al autor
        author_about = quote.find('a')
        author_link = author_about['href'] if author_about else "No disponible"

        # Obtener las etiquetas (tags)
        tags = quote.find('div', class_='tags')
        tag_list = [tag.get_text(strip=True) for tag in tags.find_all('a', class_='tag')] if tags else []

        author_links.append({
            'cita': quote_text,
            'autor': author,
            'enlace': author_link,
            'etiquetas': tag_list
        })

    print("Información de las primeras 5 citas:")
    for i, quote_info in enumerate(author_links, 1):
        print(f"\nCita {i}:")
        print(f"  Autor: {quote_info['autor']}")
        print(f"  Enlace: {quote_info['enlace']}")
        print(f"  Etiquetas: {', '.join(quote_info['etiquetas'])}")

    print("\n" + "="*50)
    print("FUNCIONALIDAD 4: NAVEGACIÓN POR LA ESTRUCTURA DEL ÁRBOL HTML")
    print("="*50)

    # Ejemplo 4.1: Navegar por padres, hijos y hermanos
    first_quote = quotes[0]

    # Obtener el padre del contenedor de la cita
    parent = first_quote.parent
    print(f"Etiqueta padre de la primera cita: {parent.name}")

    # Obtener todos los hijos directos
    children = list(first_quote.children)
    print(f"\nNúmero de elementos hijos directos: {len(children)}")
    print("Tipos de elementos hijos:")
    for i, child in enumerate(children[:5], 1):
        if hasattr(child, 'name') and child.name:
            print(f"  {i}. <{child.name}>")

    # Ejemplo 4.2: Obtener hermanos
    author_elem = first_quote.find('small', class_='author')
    if author_elem:
        # Obtener el siguiente hermano del autor
        next_sibling = author_elem.find_next_sibling()
        print(f"\nElemento siguiente al autor: <{next_sibling.name if next_sibling else 'None'}>")

    # Ejemplo 4.3: Navegar hacia arriba y abajo
    text_elem = first_quote.find('span', class_='text')
    if text_elem:
        # Navegar hacia arriba hasta encontrar el div.quote
        quote_div = text_elem.find_parent('div', class_='quote')
        print(f"\nDesde el texto, subiendo al div.quote: <{quote_div.name if quote_div else 'None'}>")

        # Navegar hacia abajo para encontrar todas las etiquetas dentro de la cita
        spans = quote_div.find_all('span')
        print(f"Número de elementos span dentro de la cita: {len(spans)}")
        print("Clases de los spans:")
        for span in spans:
            if span.has_attr('class'):
                print(f"  - {span['class']}")

    # EJEMPLO PRÁCTICO: Crear un DataFrame con toda la información
    print("\n" + "="*50)
    print("EJEMPLO PRÁCTICO: Extracción completa de datos")
    print("="*50)

    all_quotes = []

    for quote in quotes:
        text = quote.find('span', class_='text').get_text(strip=True)
        author = quote.find('small', class_='author').get_text(strip=True)
        tags = [tag.get_text(strip=True) for tag in quote.find_all('a', class_='tag')]

        all_quotes.append({
            'texto': text,
            'autor': author,
            'etiquetas': tags
        })

    # Crear DataFrame
    df = pd.DataFrame(all_quotes)
    print("\nDataFrame con información de todas las citas:")
    print(df.head())

    # Guardar en CSV
    # df.to_csv('citas_famosas.csv', index=False)
    # print("\nDatos guardados en 'citas_famosas.csv'")

else:
    print(f"Error al conectar. Código de estado: {response.status_code}")

Conexión exitosa. Código de estado: 200

FUNCIONALIDAD 1: BÚSQUEDA Y NAVEGACIÓN BÁSICA
Título de la página: Quotes to Scrape

Citas encontradas: 10

Primeras 3 citas:
1. “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
2. “It is our choices, Harry, that show what we truly are, far more than our abilities.”
3. “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”

Primer autor encontrado: Albert Einstein

FUNCIONALIDAD 2: BÚSQUEDA AVANZADA CON SELECTORES CSS Y FILTROS
Número de citas encontradas con CSS selector: 10

Citas de Albert Einstein: 3

Citas que contienen la palabra 'love': 3

FUNCIONALIDAD 3: EXTRACCIÓN Y MANIPULACIÓN DE ATRIBUTOS
Información de las primeras 5 citas:

Cita 1:
  Autor: Albert Einstein
  Enlace: /author/Albert-Einstein
  Etiquetas: change, deep-thoughts, thinking, world

Cita 2:
  Autor: J.K. Rowling
  Enlace: /author/

<ipython-input-4-6517a3072b38>:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  love_quotes = soup.find_all(text=lambda text: 'love' in text.lower() if text else False)
